In [1]:
import keras
import numpy as np

path= '/Users/duutabib/Downloads/bible.txt'
bible_text =open(path).read().lower()


Using TensorFlow backend.
/Users/duutabib/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
print('Corpus Length: ', len(bible_text))

Corpus Length:  4556377


In [3]:
len(bible_text)

4556377

In [4]:
def change_entropy(init_distrib, temp =0.5):
    # this controls the entropy of the probability distribution from which we sample  
    distrib = np.log(init_distrib)/ temp
    distrib = np.exp( distrib)
    return distrib/sum(distrib)
    

In [13]:
maxsentence_len = 30
stride = 3
sentences = []
next_characters= []

for i in range(0, len(bible_text)- maxsentence_len, stride) :
    sentences.append(bible_text[i: i + maxsentence_len])
    next_characters.append(bible_text[i+maxsentence_len])
print("Nos. of Sequences :", len(sentences))

Nos. of Sequences : 1518783


In [14]:
characters = sorted(list(set(bible_text)))
print('unique character: ' , len(characters)) 
char_indices = dict((char, characters.index(char)) for char in characters)

unique character:  50


In [15]:
print('Vectorization, precisely one-hot encoding...')
x =np.empty((len(sentences), maxsentence_len, len(characters) ),dtype=np.bool)
y =np.empty((len(sentences), len(characters)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i,t, char_indices[char] ] =1
        y[i, char_indices[next_characters[i]]] =1

Vectorization, precisely one-hot encoding...


In [16]:
#Buildingthe network

from keras import layers
lang_model = keras.models.Sequential()
lang_model.add(layers.LSTM(128, input_shape=(maxsentence_len, len(characters))))
lang_model.add(layers.Dense(len(characters), activation='softmax'))
lang_model.summary()
optimizer = keras.optimizers.RMSprop(lr=0.01)
lang_model.compile(loss= 'categorical_crossentropy', optimizer=optimizer)

def gen_sample(predicts, temp):
    predicts = np.asarray(predicts).astype('float64')  
    predicts = change_entropy(predicts, temp)
    probs = np.random.multinomial(1, predicts, 1)
    return np.argmax(probs)
    


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                6450      
Total params: 98,098
Trainable params: 98,098
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import random 
import sys

for epoch in range(1, 20):
    print('epoch ', epoch)
    lang_model.fit(x, y, batch_size=128, epochs=1 )
    start_index = random.randint(0, len(bible_text)- maxsentence_len-1)
    generated_text = bible_text[start_index: start_index + maxsentence_len]
    print(' ---- Generating with seed: "' + generated_text + '" \n')
    
    for temp in np.linspace(0.1, 1.5, 5):
        print('-----temp :', temp)
        sys.stdout.write(generated_text)
        for i in range(400):
            sampled = np.zeros((1, maxsentence_len, len(characters)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] =1
                predicts =lang_model.predict(sampled, verbose=0)[0]
                next_index = gen_sample(predicts, temp)
                next_char = characters[next_index] 
                generated_text += next_char
                generated_text = generated_text[1:]
                sys.stdout.write(next_char)

epoch  1
Epoch 1/1
  19072/1518783 [..............................] - ETA: 9:13 - loss: 2.4924